<br>
<img src="https://images.even3.com.br/ONieRmTALZ2ETt2LfoUY4_IopIU=/fit-in/250x250/smart/even3.blob.core.windows.net/logos/UFCSPA-logotipo-1.19fc99573cf34163a727.png" style="width: 100px;"/>
<center>Aprendizado profundo para Visão Computacional I
    <br>
    Prof. Viviane Botelho</center>

# <center><font color='blue'>Ajuste de hiperparâmetros</font></center>

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras as ke
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
import cv2
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import os, random

from scipy.spatial.distance import pdist, squareform

%config Completer.use_jedi = False

import warnings
warnings.simplefilter("ignore")


# Evitar warnings do tensorflow
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

# Dados

### Banco de dados: Fashion MNIST
Contém 70 mil imagens (60mil oara treinamento e 10 mil para teste) imagens em tons de cinza em 10 categorias.
https://www.tensorflow.org/tutorials/keras/classification?hl=pt-br

<img src="https://uploaddeimagens.com.br/images/003/297/013/full/Capturar.PNG?1624226287" style="width: 400px;"/>

**Objetivo: Identificar a classe da roupa**

### <font color='red'> Obs: no exemplo a seguir NÃO iremos usar validação cruzada!</font>

<br>

### Preparação dos dados 

In [ ]:
fashion_mnist = ke.datasets.fashion_mnist
(X2, Y2), (X_teste, Y_teste) = fashion_mnist.load_data()
X_treino, X_val, Y_treino, Y_val = train_test_split(X2, Y2, test_size=0.2,random_state=20,shuffle=True,stratify=Y2)

# codificacao da saída (one-hot encoding)
Y_treino=ke.utils.to_categorical(Y_treino)
Y_teste=ke.utils.to_categorical(Y_teste)
Y_val=ke.utils.to_categorical(Y_val)

# Normalização das entradas
X_treino = X_treino/255.0
X_teste= X_teste/255.0
X_val= X_val/255.0

## Efeito dos hiperparâmetros

In [ ]:
model = ke.Sequential()
model.add(ke.layers.Flatten(input_shape=(28, 28,)))
#model.add(ke.layers.Dropout(0.2))
model.add(ke.layers.Dense(90, activation='relu'))
#model.add(ke.layers.BatchNormalization())
#model.add(ke.layers.Dropout(0.2))
model.add(ke.layers.Dense(10, activation='softmax'))


#opt = ke.optimizers.Adam(learning_rate=1e-5)
# opt = ke.optimizers.Adam(learning_rate= ke.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=0.1,
#     decay_steps=10,
#     decay_rate=0.9))


model.compile(optimizer='Adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# fixar estado de inicialização. Melhorar a reprodutibilidade
i=543
os.environ['PYTHONHASHSEED']=str(i)
tf.random.set_seed(i)
np.random.seed(i)
random.seed(i)

history=model.fit(x=X_treino, y=Y_treino,
          epochs=20,batch_size=2000,
          validation_data=(X_val,Y_val),verbose=False)

# análise dos resultados
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(history.history["loss"], label="treino")
plt.plot(history.history["val_loss"], label="validacao")
plt.xlabel("epocas")
plt.ylabel("Loss")
plt.legend();
plt.subplot(1,2,2)
plt.plot(history.history["accuracy"], label="treino")
plt.plot(history.history["val_accuracy"], label="validacao")
plt.xlabel("epocas")
plt.ylabel("acurácia")
plt.legend();

Y_pred_prob= model.predict(X_teste)
Y_pred_teste = np.argmax(Y_pred_prob,axis=1)  # resultado numerico
Y_teste_num= np.argmax(Y_teste,axis=1)   # retorna para numerico (era categorico)

cmat=confusion_matrix(Y_pred_teste,Y_teste_num)
cm_df = pd.DataFrame(cmat) 

plt.figure()
ax= plt.subplot()
sns.heatmap(cm_df,annot=True, cmap='Reds',fmt="d",cbar=False)

# Métodos automáticos de ajuste de hiperparâmetros

<img src="https://developer.qualcomm.com/sites/default/files/attachments/learning_resources_03-05.png" width="600px;"/>



    
    
    

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import make_scorer 
from sklearn.metrics import  balanced_accuracy_score

1) Criar o modelo, colocando como argumento opcional o parâmetro a ser variado

In [ ]:
def model_grid(n,learn_rate):
    model = ke.Sequential()
    model.add(ke.layers.Flatten(input_shape=(28, 28,)))
    model.add(ke.layers.Dense(n, activation='relu'))
    model.add(ke.layers.BatchNormalization())
    model.add(ke.layers.Dropout(0.5))
    model.add(ke.layers.Dense(10, activation='softmax'))
    opt = ke.optimizers.Adam(lr=learn_rate)
    model.compile(optimizer=opt, 
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model


2) Definindo os parametros de busca

In [ ]:
lr_grid = [0.1,0.05]
n_grid = [2,90]

param_grid = dict(learn_rate=lr_grid, n=n_grid)

3) Cria e treina o modelo

#### <font color='red'> Obs:GridSearchCV já permite a realização automática da validação cruzada!</font>

In [ ]:
model = KerasClassifier(build_fn=model_grid, epochs=20, batch_size=2000, verbose=0)

grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=3,refit=False)
grid_result = grid.fit(X2, Y2)

5) Avaliando os resultados

In [ ]:
df=pd.DataFrame(grid_result.cv_results_)
df

<font color='red'>Importante: sempre retreinar o modelo com os melhores hiperparâmetros selecionados!</font></center>